<a href="https://colab.research.google.com/github/jimin-fundamental/Tamtam-AI/blob/main/tamtam_AI_prediction_real_data%2Bdata_procedure%2B%EC%A0%95%EA%B7%9C%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install autogluon  # 라이브러리 설치 (Colab에서 한 번 실행 필요)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 19.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. 

### 1️⃣ 구글 드라이브 마운트 & 데이터 불러오기

In [51]:
from google.colab import auth
import gspread
from google.auth import default
import pandas as pd
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame

# 구글 드라이브 인증 및 마운트
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Google Spreadsheet 불러오기 (스프레드시트 ID 입력 필요)
spreadsheet_url1 = "https://docs.google.com/spreadsheets/d/19eBMHXQVWEo_OfG46LBTFZMH-sas2p3NwE7_cZNBLrM/edit?gid=447779875#gid=447779875"
spreadsheet_url2 = "https://docs.google.com/spreadsheets/d/14kCL7_IOCFf5K-FgHaC7_f9ar4-9biadwBASdsFiTa4/edit?gid=938858036#gid=938858036"
spreadsheet_url3 = "https://docs.google.com/spreadsheets/d/1HLU3UqdMJQ6Q9rbWiKP6nfMRID-Mzvr89sHLgcL82kw/edit?gid=573315909#gid=573315909"
sheet = gc.open_by_url(spreadsheet_url1).sheet1

# 데이터를 DataFrame으로 변환
data = sheet.get_all_records()
df = pd.DataFrame(data)

# ✅ 데이터 확인
print("---------1차------------")
print(df.head())
print(df.columns)


---------1차------------
           날짜     시간  타입    대분류     소분류                내용      금액   화폐  \
0  2025-02-10  03:20  지출  주거/통신     가스비        도시가스02x201 -119460  KRW   
1  2025-02-09  15:44  지출     교통    대중교통  모바일티머니 후불무승인_지하철   -1400  KRW   
2  2025-02-09  15:42  지출     교통    대중교통  모바일티머니 후불무승인_지하철   -1400  KRW   
3  2025-02-09  14:05  지출     식비      고기              도성한우   -9500  KRW   
4  2025-02-09  12:33  지출  카페/간식  도넛/핫도그     크리스피크림도넛 가좌역점   -5766  KRW   

                  결제수단 메모  
0       우리 SUPER주거래 통장     
1  케이뱅크 교통 전용(MY K-패스)     
2  케이뱅크 교통 전용(MY K-패스)     
3       네이버페이 간편결제(머니)     
4       네이버페이 간편결제(머니)     
Index(['날짜', '시간', '타입', '대분류', '소분류', '내용', '금액', '화폐', '결제수단', '메모'], dtype='object')


### 🥈 데이터 처리

2-1. 날짜랑 지출 데이터만 가져오기

In [52]:
# 모든 행과 열 출력하도록
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [53]:
import pandas as pd

# 날짜와 시간을 합쳐서 새로운 timestamp 컬럼 생성
df["timestamp"] = pd.to_datetime(df["날짜"] + " " + df["시간"])

# 불필요한 컬럼 제거 (날짜, 시간, 소분류, 내용, 화폐, 결제수단, 메모 제거)
df = df.drop(columns=["날짜", "시간", "소분류", "내용", "화폐", "결제수단", "메모"])

# ✅ 1차 데이터 확인
print("---------1차------------")
print(df.head())
print(df.dtypes)  # timestamp가 datetime 형식인지 확인

# # ✅ 2024-08-12 이후 데이터만 필터링
# df = df[df["timestamp"] >= "2024-08-12"]

# 타입 = "지출" 데이터만 가져오기
df = df[df["타입"] == "지출"]

# 금액의 음수(-) 제거 (숫자로 변환 후 절댓값 적용)
df["금액"] = df["금액"].abs()

# "타입" 컬럼 제거
df = df.drop(columns=["타입"])

# 컬럼명 변경 (TimeSeriesDataFrame 요구사항에 맞게)
df = df.rename(columns={"대분류": "category"})  # "대분류" → "category" 변경

# 여기서 원하는 카테고리만 유지하고 나머지는 drop
valid_categories = [
    "식비", "카페/간식", "교통", "온라인쇼핑",
    "술/유흥", "생활", "패션/쇼핑", "뷰티/미용",
    "문화/여가", "여행/숙박"
]
df = df[df["category"].isin(valid_categories)]


# ✅ 최종 데이터 확인
print("---------필요한 칼럼만 남음------------")
print(df.head())
print(df.columns)

# ✅ 새로운 이름으로 파일 저장 (CSV & Excel)
# df.to_csv("processed_data.csv", index=False, encoding="utf-8-sig")  # CSV 저장
df.to_excel("processed_data.xlsx", index=False)  # Excel 저장

print("✅ 파일 저장 완료: processed_data.xlsx")

---------1차------------
   타입    대분류      금액           timestamp
0  지출  주거/통신 -119460 2025-02-10 03:20:00
1  지출     교통   -1400 2025-02-09 15:44:00
2  지출     교통   -1400 2025-02-09 15:42:00
3  지출     식비   -9500 2025-02-09 14:05:00
4  지출  카페/간식   -5766 2025-02-09 12:33:00
타입                   object
대분류                  object
금액                    int64
timestamp    datetime64[ns]
dtype: object
---------필요한 칼럼만 남음------------
  category    금액           timestamp
1       교통  1400 2025-02-09 15:44:00
2       교통  1400 2025-02-09 15:42:00
3       식비  9500 2025-02-09 14:05:00
4    카페/간식  5766 2025-02-09 12:33:00
5    카페/간식   134 2025-02-09 12:33:00
Index(['category', '금액', 'timestamp'], dtype='object')
✅ 파일 저장 완료: processed_data.xlsx


In [54]:
# ✅ 저장된 파일 불러오기 (Excel 파일)
df = pd.read_excel("processed_data.xlsx")

# ✅ 결측값 확인 (info() 사용)
print("--------- 데이터 정보 & 결측값 확인 ------------")
print(df.info())  # 데이터 타입 및 결측값 확인
print("\n결측값 개수:\n", df.isnull().sum())  # 결측값 개수 출력

--------- 데이터 정보 & 결측값 확인 ------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865 entries, 0 to 864
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   category   865 non-null    object        
 1   금액         865 non-null    int64         
 2   timestamp  865 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 20.4+ KB
None

결측값 개수:
 category     0
금액           0
timestamp    0
dtype: int64


데이터를 주 단위로 자르기

In [55]:
# ✅ 주 단위 변환 (주의 첫날로 변환 - Datetime 형식)
df["week"] = df["timestamp"].dt.to_period("W").apply(lambda x: x.start_time)

# ✅ timestamp 컬럼 제거
df = df.drop(columns=["timestamp"])

# 컬럼명 변경 (TimeSeriesDataFrame 요구사항에 맞게)
df = df.rename(columns={"week": "timestamp", "금액": "target", "category": "item_id"})

# 카테고리별 주별 소비 금액 집계 (불필요한 컬럼 제거)
weekly_df = df.groupby(["item_id", "timestamp"], as_index=False).agg({
    "target": "sum"  # 카테고리별 주별 소비 금액 합산
})

# 연속적인 주별 데이터 생성 (누락된 주 채우기)
start_date = weekly_df["timestamp"].min()  # 최소 날짜
end_date = weekly_df["timestamp"].max()  # 최대 날짜
all_weeks = pd.date_range(start=start_date, end=end_date, freq="7D")  # 7일 간격

# 카테고리별로 모든 주를 포함하는 데이터프레임 생성
all_categories = weekly_df["item_id"].unique()  # 고유 카테고리 목록
all_combinations = pd.MultiIndex.from_product([all_categories, all_weeks], names=["item_id", "timestamp"])
all_weeks_df = pd.DataFrame(index=all_combinations).reset_index()

# 기존 데이터와 병합하여 없는 주는 0으로 채우기
weekly_df_full = pd.merge(all_weeks_df, weekly_df, on=["item_id", "timestamp"], how="left")
weekly_df_full["target"] = weekly_df_full["target"].fillna(0)  # 없는 주는 0으로 채움

# 데이터 확인
print("✅ 주별 집계 데이터 확인 (누락된 주 포함)")
print(weekly_df_full)
print("✅ 컬럼 목록:", weekly_df_full.columns)
print("✅ 데이터 크기:", weekly_df_full.shape)

print("✅ 컬럼 목록:", weekly_df.columns)
print("✅ 데이터 크기:", weekly_df.shape)

✅ 주별 집계 데이터 확인 (누락된 주 포함)
    item_id  timestamp  target
0        교통 2024-07-08       0
1        교통 2024-07-15       0
2        교통 2024-07-22       0
3        교통 2024-07-29   10000
4        교통 2024-08-05       0
5        교통 2024-08-12   10000
6        교통 2024-08-19  129200
7        교통 2024-08-26  250840
8        교통 2024-09-02   45200
9        교통 2024-09-09   66900
10       교통 2024-09-16  152900
11       교통 2024-09-23   11700
12       교통 2024-09-30    7500
13       교통 2024-10-07  160100
14       교통 2024-10-14   30000
15       교통 2024-10-21       0
16       교통 2024-10-28    5000
17       교통 2024-11-04   12100
18       교통 2024-11-11   17000
19       교통 2024-11-18   11900
20       교통 2024-11-25    6300
21       교통 2024-12-02    3000
22       교통 2024-12-09       0
23       교통 2024-12-16    1700
24       교통 2024-12-23   27700
25       교통 2024-12-30   30700
26       교통 2025-01-06  104300
27       교통 2025-01-13   18860
28       교통 2025-01-20   61190
29       교통 2025-01-27   88200
30       교통 2

In [56]:
from autogluon.timeseries import TimeSeriesDataFrame

# ✅ 변환된 데이터를 새로운 엑셀 파일로 저장
weekly_df_full.to_excel("processed_data_weekly.xlsx", index=False)  # Excel 파일 저장
print("✅ 파일 저장 완료: processed_data_weekly.xlsx")

✅ 파일 저장 완료: processed_data_weekly.xlsx


### 3️⃣ 탐색적 자료 분석 및 시각화

In [57]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis, zscore

# ======================================
# 0. 처리된 데이터 불러오기
# ======================================
# 저장된 주별 데이터 파일에서 데이터를 읽어옴
weekly_budget_ts_df = pd.read_excel("processed_data_weekly.xlsx")
print("✅ processed_data_weekly.xlsx 파일 로드 완료")
print(weekly_budget_ts_df.head())

# ======================================
# 1. 전체 데이터 기본 통계 계산
# ======================================
target_mean = weekly_budget_ts_df["target"].mean()    # 평균
target_std = weekly_budget_ts_df["target"].std()       # 표준편차
target_min = weekly_budget_ts_df["target"].min()       # 최소값
target_max = weekly_budget_ts_df["target"].max()       # 최대값
target_median = weekly_budget_ts_df["target"].median() # 중앙값

print(f"\n전체 데이터 - 평균: {target_mean:.2f}, 표준편차: {target_std:.2f}")
print(f"최소값: {target_min}, 최대값: {target_max}, 중앙값: {target_median}")

✅ processed_data_weekly.xlsx 파일 로드 완료
  item_id  timestamp  target
0      교통 2024-07-08       0
1      교통 2024-07-15       0
2      교통 2024-07-22       0
3      교통 2024-07-29   10000
4      교통 2024-08-05       0

전체 데이터 - 평균: 41115.28, 표준편차: 94197.35
최소값: 0, 최대값: 727200, 중앙값: 6900.0


In [58]:
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis

# ======================================
# 2. 카테고리별 통계값 계산
# ======================================
stats_df = weekly_budget_ts_df.groupby("item_id")["target"].agg(
    mean="mean",         # 평균
    std="std",           # 표준편차
    min="min",           # 최소값
    max="max",           # 최대값
    median="median",     # 중앙값
    variance="var",      # 분산
).reset_index()

print("\n각 카테고리별 기본 통계값:")
print(stats_df)


각 카테고리별 기본 통계값:
  item_id  mean    std  min     max  median    variance
0      교통 41351  60259    0  250840   12100  3631097858
1   문화/여가 17519  31525    0  118000    5000   993838946
2   뷰티/미용 17786  71934    0  400000       0  5174532192
3      생활 95088 166379    0  722530   30530 27682040666
4    술/유흥  8598  27049    0  139100       0   731661305
5      식비 88082  97680    0  380850   55780  9541393272
6   여행/숙박 23856  80194    0  406946       0  6431063205
7   온라인쇼핑 79844  97642    0  411061   50430  9533997732
8   카페/간식 36520 129197    0  727200    6800 16691981620
9   패션/쇼핑  2508   6481    0   31000       0    42000391


In [59]:
# 예시: 원본 데이터 df가 있고, 'item_id'와 'target' 컬럼이 있다고 가정
# 각 카테고리별 0의 비율(zero_rate)을 계산
zero_stats = weekly_budget_ts_df.groupby('item_id')['target'].apply(lambda x: (x == 0).mean()).reset_index()
zero_stats = zero_stats.rename(columns={'target': 'zero_rate'})
print(zero_stats)

# 기존 통계 데이터(stats_df)에 위 통계치를 병합
stats_df = stats_df.merge(zero_stats, on='item_id', how='left')

# 제외할 카테고리 목록
exclude_categories = ['미분류']

# 유효 카테고리 선택:
# - NaN이 없고 std > 0,
# - 제외할 카테고리가 아니며,
# - 0의 비율(zero_rate)이 50% 미만인 카테고리
stats_df_valid = stats_df[
    (~stats_df['item_id'].isin(exclude_categories)) &
    (stats_df['std'] > 0) &
    (stats_df['zero_rate'] < 0.5)
]
print("\n유효 카테고리 (조건 적용 후):")
print(stats_df_valid)

# 표준편차 임계치 적용 (예: std < 100,000) 및 "식비" 항상 포함
std_threshold = 150000  # 데이터 특성에 따라 조정
selected_categories = stats_df_valid[stats_df_valid['std'] < std_threshold]['item_id'].tolist()

if "식비" not in selected_categories:
    selected_categories.append("식비")

print("\n예측 대상으로 선택된 카테고리:")
print(selected_categories)

  item_id  zero_rate
0      교통          0
1   문화/여가          0
2   뷰티/미용          1
3      생활          0
4    술/유흥          1
5      식비          0
6   여행/숙박          1
7   온라인쇼핑          0
8   카페/간식          0
9   패션/쇼핑          1

유효 카테고리 (조건 적용 후):
  item_id  mean    std  min     max  median    variance  zero_rate
0      교통 41351  60259    0  250840   12100  3631097858          0
1   문화/여가 17519  31525    0  118000    5000   993838946          0
3      생활 95088 166379    0  722530   30530 27682040666          0
5      식비 88082  97680    0  380850   55780  9541393272          0
7   온라인쇼핑 79844  97642    0  411061   50430  9533997732          0
8   카페/간식 36520 129197    0  727200    6800 16691981620          0

예측 대상으로 선택된 카테고리:
['교통', '문화/여가', '식비', '온라인쇼핑', '카페/간식']


In [60]:
from scipy.stats import skew, kurtosis, zscore
import numpy as np

# ======================================
# 3. 이상치 보정
# ======================================
# 선택된 카테고리의 데이터만 사용 (복사본 생성)
df = weekly_budget_ts_df[weekly_budget_ts_df['item_id'].isin(selected_categories)].copy()
df = df.sort_values(["item_id", "timestamp"]).reset_index(drop=True)

# --------------------------------------
# 3-1. 이동평균 기반 이상치 보정 (4주 윈도우)
# --------------------------------------
window_size = 4  # 4주 이동 평균
df["rolling_mean"] = df["target"].rolling(window=window_size, min_periods=1).mean()
df["rolling_std"] = df["target"].rolling(window=window_size, min_periods=1).std()

# 이동 평균 ± 3 * 표준편차 벗어나면 이상치로 간주
df["is_outlier_ma"] = (df["target"] > df["rolling_mean"] + 1.8 * df["rolling_std"]) | \
                      (df["target"] < df["rolling_mean"] - 1.8 * df["rolling_std"])

# 이동평균 기반 이상치 판별 후, 보정하기 전에 이상치 행 출력하기
print("이동평균 기반 이상치 판별 결과 (보정 전):")
print(df[df["is_outlier_ma"]][["item_id", "timestamp", "target", "rolling_mean", "rolling_std"]])

# 보존용으로 원본 데이터를 복사 (이동평균 보정 전)
df["target_original_ma"] = df["target"]

# 이상치 보정: 해당 값을 이동 평균 값으로 대체
df.loc[df["is_outlier_ma"], "target"] = df["rolling_mean"]

print("\n[이동평균 기반 이상치 보정] 일부 결과:")
print(df[["item_id", "timestamp", "target", "rolling_mean", "rolling_std", "is_outlier_ma"]].head(10))


# --------------------------------------
# 3-2. IQR 및 Z-score 기반 이상치 보정
# --------------------------------------
# IQR 기반 이상치 감지
Q1 = df["target"].quantile(0.25)
Q3 = df["target"].quantile(0.75)
IQR = Q3 - Q1
df["is_outlier_iqr"] = (df["target"] > (Q3 + 1.5 * IQR)) | (df["target"] < (Q1 - 1.5 * IQR))

# Z-score 기반 이상치 감지
df["z_score"] = np.abs(zscore(df["target"]))
df["is_outlier_z"] = df["z_score"] > 3

# IQR와 Z-score 조건 모두 만족하면 이상치로 판단
df["is_outlier"] = df["is_outlier_iqr"] | df["is_outlier_z"]

# IQR 및 Z-score 기반 이상치 판별 후, 보정하기 전에 이상치 행 출력하기
print("\nIQR & Z-score 기반 이상치 판별 결과 (보정 전):")
print(df[["item_id", "timestamp", "target", "is_outlier_iqr", "is_outlier_z"]].head(10))

# 보존용으로 원본 데이터를 복사 (IQR & Z-score 보정 전)
df["target_original_iqr"] = df["target"]

# 이상치 보정: 이상치 값을 다시 이동평균 값으로 대체
df.loc[df["is_outlier"], "target"] = df["rolling_mean"]


print("\n[이상치 보정] 전체 결과:")
print(df[["item_id", "target", "is_outlier", "target_original_iqr"]])


final_df = df[["item_id", "timestamp", "target"]].copy()

# 최종 결과 확인
print("\n최종 처리된 데이터 (상위 10행):")
print(final_df.head())


이동평균 기반 이상치 판별 결과 (보정 전):
Empty DataFrame
Columns: [item_id, timestamp, target, rolling_mean, rolling_std]
Index: []

[이동평균 기반 이상치 보정] 일부 결과:
  item_id  timestamp  target  rolling_mean  rolling_std  is_outlier_ma
0      교통 2024-07-08       0             0          NaN          False
1      교통 2024-07-15       0             0            0          False
2      교통 2024-07-22       0             0            0          False
3      교통 2024-07-29   10000          2500         5000          False
4      교통 2024-08-05       0          2500         5000          False
5      교통 2024-08-12   10000          5000         5774          False
6      교통 2024-08-19  129200         37300        61448          False
7      교통 2024-08-26  250840         97510       117871          False
8      교통 2024-09-02   45200        108810       107079          False
9      교통 2024-09-09   66900        123035        92343          False

IQR & Z-score 기반 이상치 판별 결과 (보정 전):
  item_id  timestamp  target  is_outlier_

<ipython-input-60-6a2e0db1e54b>:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 97510.   128960.    83050.    72900.   150950.   224562.5  157696.75
 130923.5  192023.    65673.75  95708.5  161383.   204300.  ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df["is_outlier"], "target"] = df["rolling_mean"]


In [61]:
# ✅ 변환된 데이터를 새로운 엑셀 파일로 저장
final_df.to_excel("fullyprocessed_data_weekly.xlsx", index=False)  # Excel 파일 저장
print("✅ 파일 저장 완료: fullyprocessed_data_weekly.xlsx")

✅ 파일 저장 완료: fullyprocessed_data_weekly.xlsx


## 모델(DirectTabularRegressionForecaster) 실행 전 데이터 추가 처리

데이터 분할 ⇒ Train-Test Split (Temporal Split)

In [62]:
import pandas as pd

# 1. 데이터 불러오기 및 timestamp 컬럼 datetime 변환
df = pd.read_excel("fullyprocessed_data_weekly.xlsx")
df["timestamp"] = pd.to_datetime(df["timestamp"])

# 2. 전역 주차(글로벌 타임스탬프 범위) 생성
global_start = df["timestamp"].min()  # 예: 2024-08-12
global_end = df["timestamp"].max()      # 예: 2025-02-10
global_weeks = pd.date_range(start=global_start, end=global_end, freq="7D")
length = len(global_weeks)
print(f"전역 주차 길이: {length}")

# 3. 비율 기반 데이터 분할 설정
train_size = int(length * 1 / 5)

# 4. 카테고리별로 전역 주차 기준 재인덱싱 후 슬라이싱
train_test_splits = {}

for category in df["item_id"].unique():
    # 1. 해당 카테고리 원본 데이터 추출
    cat_df = df[df["item_id"] == category].set_index("timestamp")

    # 2. 전역 주차 기반으로 DataFrame 생성 (기본 target=0, item_id=category)
    reindexed = pd.DataFrame(index=global_weeks)
    reindexed["target"] = 0
    reindexed["item_id"] = category

    # 3. 원본 데이터가 존재하는 시점을 표시하는 플래그 생성
    #    글로벌 인덱스에 해당하는 부분만 True로 표시
    reindexed["has_data"] = reindexed.index.isin(cat_df.index)

    # 4. 기존 원본 데이터로 업데이트
    reindexed.update(cat_df)

    # 5. 인덱스 재설정 및 timestamp 컬럼 복원
    reindexed = reindexed.reset_index().rename(columns={"index": "timestamp"})

    # 6. has_data가 True인 행(즉, 원본 데이터가 있는 주차만)을 선택
    reindexed = reindexed[reindexed["has_data"]].reset_index(drop=True)

    # 이제 빈 주(혹은 달)은 이미 제거된 상태에서 슬라이싱 진행
    train_1 = reindexed.iloc[:train_size+2]
    print(len(train_1))
    test_1 = reindexed.iloc[train_size+2:train_size+3]

    train_2 = reindexed.iloc[train_size + 3:train_size*3-5]
    print(len(train_2))
    test_2 = reindexed.iloc[train_size*3-5:train_size*3-4]
    train_3 = reindexed.iloc[train_size*3+9:length-2]
    print(len(train_3))
    test_3 = reindexed.iloc[length-2:length]

    train_test_splits[category] = {
        "train_1": train_1,
        "test_1": test_1,
        "train_2": train_2,
        "test_2": test_2,
        "train_3": train_3,
        "test_3": test_3,
    }

    print(f"📌 카테고리: {category}")
    print(f"Train 1: {train_1['timestamp'].min()} ~ {train_1['timestamp'].max()}")
    print(f"Test 1: {test_1['timestamp'].min()} ~ {test_1['timestamp'].max()}")
    print(f"Train 2: {train_2['timestamp'].min()} ~ {train_2['timestamp'].max()}")
    print(f"Test 2: {test_2['timestamp'].min()} ~ {test_2['timestamp'].max()}")
    print(f"Train 3: {train_3['timestamp'].min()} ~ {train_3['timestamp'].max()}")
    print(f"Test 3: {test_3['timestamp'].min()} ~ {test_3['timestamp'].max()}")
    print("-" * 50)


# 5. 엑셀 파일로 저장 (엑셀 시트 이름에서 "/"는 "-"로 대체)
with pd.ExcelWriter("split_train_test.xlsx") as writer:
    for category, splits in train_test_splits.items():
        sanitized_category = category.replace("/", "-")
        for key, split_data in splits.items():
            split_data.to_excel(writer, sheet_name=f"{sanitized_category}_{key}", index=False)

print("✅ 데이터 저장 완료: split_train_test.xlsx")


전역 주차 길이: 31
8
4
2
📌 카테고리: 교통
Train 1: 2024-07-08 00:00:00 ~ 2024-08-26 00:00:00
Test 1: 2024-09-02 00:00:00 ~ 2024-09-02 00:00:00
Train 2: 2024-09-09 00:00:00 ~ 2024-09-30 00:00:00
Test 2: 2024-10-07 00:00:00 ~ 2024-10-07 00:00:00
Train 3: 2025-01-13 00:00:00 ~ 2025-01-20 00:00:00
Test 3: 2025-01-27 00:00:00 ~ 2025-02-03 00:00:00
--------------------------------------------------
8
4
2
📌 카테고리: 문화/여가
Train 1: 2024-07-08 00:00:00 ~ 2024-08-26 00:00:00
Test 1: 2024-09-02 00:00:00 ~ 2024-09-02 00:00:00
Train 2: 2024-09-09 00:00:00 ~ 2024-09-30 00:00:00
Test 2: 2024-10-07 00:00:00 ~ 2024-10-07 00:00:00
Train 3: 2025-01-13 00:00:00 ~ 2025-01-20 00:00:00
Test 3: 2025-01-27 00:00:00 ~ 2025-02-03 00:00:00
--------------------------------------------------
8
4
2
📌 카테고리: 식비
Train 1: 2024-07-08 00:00:00 ~ 2024-08-26 00:00:00
Test 1: 2024-09-02 00:00:00 ~ 2024-09-02 00:00:00
Train 2: 2024-09-09 00:00:00 ~ 2024-09-30 00:00:00
Test 2: 2024-10-07 00:00:00 ~ 2024-10-07 00:00:00
Train 3: 2025-01-13 00:

<ipython-input-62-900b8e9ef318>:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[  8000.    16000.    22500.    28000.    52900.    47300.    72900.
 150950.   100000.    77200.   224562.5  129060.        0.    23500.
  41700.    27500.    76300.    28327.    50690.   104700.    55780.
  36000.    64295.        0.   100000.   100300.    86487.   157696.75
  70000.    25863.   129690.  ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  reindexed.update(cat_df)
<ipython-input-62-900b8e9ef318>:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 13973.    45200.    78650.    60040.    50430.    63945.    10400.
      0.    12949.    99684.   130923.5  100460.    95500.   192023.
  76160.     6500.        0.    10986.    41700.    90383.    29920.
  19175.    10000.     3950.    65673.75  45300.    62635.  

### 3 모델을 활용하여 학습 후 그 다음주부터 예측

### (1) 통계적 모델(Prophet) 적용

In [63]:
print(df)

    item_id  timestamp  target
0        교통 2024-07-08       0
1        교통 2024-07-15       0
2        교통 2024-07-22       0
3        교통 2024-07-29   10000
4        교통 2024-08-05       0
5        교통 2024-08-12   10000
6        교통 2024-08-19  129200
7        교통 2024-08-26   97510
8        교통 2024-09-02   45200
9        교통 2024-09-09   66900
10       교통 2024-09-16  128960
11       교통 2024-09-23   11700
12       교통 2024-09-30    7500
13       교통 2024-10-07   83050
14       교통 2024-10-14   30000
15       교통 2024-10-21       0
16       교통 2024-10-28    5000
17       교통 2024-11-04   12100
18       교통 2024-11-11   17000
19       교통 2024-11-18   11900
20       교통 2024-11-25    6300
21       교통 2024-12-02    3000
22       교통 2024-12-09       0
23       교통 2024-12-16    1700
24       교통 2024-12-23   27700
25       교통 2024-12-30   30700
26       교통 2025-01-06  104300
27       교통 2025-01-13   18860
28       교통 2025-01-20   61190
29       교통 2025-01-27   88200
30       교통 2025-02-03   19600
31    문화

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error

pd.set_option('display.float_format', '{:.0f}'.format)

# -------------------------------
# 평가 지표 계산 함수들
# -------------------------------

# SMAPE 계산 함수
def smape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_true - y_pred)
    smape_values = np.where(denominator == 0, 0, diff / denominator)
    return np.mean(smape_values) * 100

# MASE 계산 함수 (훈련 데이터를 기준으로 스케일링)
def mase(training_series, y_true, y_pred):
    training_series = np.array(training_series)
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    scale = np.mean(np.abs(training_series[1:] - training_series[:-1]))
    if scale == 0:
        return np.nan
    return mean_absolute_error(y_true, y_pred) / scale

# -------------------------------
# Prophet 예측 및 평가
# -------------------------------
forecasts = {}
metrics = {}
results_list = []  # 전체 예측 결과 저장용 리스트

# y: 실제값 (ground truth)
# yhat: Prophet이 예측한 값 (predicted value)

for category, splits in train_test_splits.items():
    print(f"📌 카테고리: {category}")
    category_metrics = {}

    for step in range(1, 4):  # Step 1, 2, 3 예측
        train_df = splits[f"train_{step}"].copy()
        test_df = splits[f"test_{step}"].copy()
        # Prophet에서는 시간 컬럼명이 'ds'와 값 컬럼명이 'y'여야 함
        train_df = train_df.rename(columns={"timestamp": "ds", "target": "y"})
        test_df = test_df.rename(columns={"timestamp": "ds", "target": "y"})

         # 로그 변환: 학습 데이터에 log1p 적용 (0 이상의 값이라 가정)
        train_df["y"] = np.log1p(train_df["y"])

        # Prophet 모델 생성 (changepoint_prior_scale을 0.7로 설정하여 최근 데이터 반영 강화)
        model = Prophet(weekly_seasonality=True, yearly_seasonality=False,
                        changepoint_prior_scale=0.7)
        model.fit(train_df)

        # 테스트 기간에 대한 예측 준비
        future = test_df[['ds']].copy()
        forecast = model.predict(future)
        forecast['yhat'] = forecast['yhat'].clip(lower=0)

        # 예측 결과는 로그 스케일이므로 역변환 (expm1)
        forecast['yhat'] = np.expm1(forecast['yhat'])
        forecast['yhat'] = forecast['yhat'].clip(lower=0)

        # 테스트 데이터와 예측 결과 병합
        result = pd.merge(test_df, forecast[['ds', 'yhat']], on='ds', how='left')
        result["Category"] = category
        result["Step"] = f"Step{step}"

        # --------------------------------------------
        # 이동 평균 기반 이상치 보정 단계
        # --------------------------------------------
        # 테스트 데이터 시작 날짜를 찾음 (예측 시점 이전의 데이터를 기준으로 window 통계 계산)
        test_start_date = test_df['ds'].min()  # 테스트 데이터의 첫 날짜

        # 전체 데이터(df)의 가장 이른 날짜
        earliest_date = train_df['ds'].min()

        window_start_date = test_start_date - pd.Timedelta(weeks=8)


        # 만약 8주 전이 전체 데이터의 시작보다 이전이면, 가능한 가장 이른 날짜로 설정합니다.
        if window_start_date < earliest_date:
            window_start_date = earliest_date

        # 사용 가능한 기간 계산: 테스트 시작일부터 window_start_date까지
        available_period = test_start_date - window_start_date

        # 사용 가능한 기간이 1주 미만이면 보정을 건너뛰고, 원래 예측값을 그대로 사용합니다.
        if available_period < pd.Timedelta(weeks=1):
            result['yhat_adjusted'] = result['yhat']
            print("사용 가능한 기간이 1주 미만이므로 예측값 보정을 건너뜁니다.")
        else:
            # window 기간 내의 실제 사용량 데이터 선택
            df_window = df[(df['timestamp'] >= window_start_date) & (df['timestamp'] < test_start_date)]

            # window 내 최소/최대값 계산
            window_min = df_window['target'].min()
            window_max = df_window['target'].max()

            # 계산된 최소/최대값을 result에 상수로 추가
            result['window_min'] = window_min
            result['window_max'] = window_max

            # 예측값이 window 내 최소값보다 작으면 최소값, 최대값보다 크면 최대값으로 보정
            result['yhat_adjusted'] = result['yhat'].clip(lower=result['window_min'], upper=result['window_max'])

            print(f"사용 기간: {window_start_date} ~ {test_start_date} (총 {available_period.days}일)")
            print(f"계산된 window_min: {window_min}, window_max: {window_max}")


        # 결과 확인(시간, 정답, 예측값, 조정된 예측값, 이동최대, 이동최소)
        print(result[['ds', 'y', 'yhat', 'yhat_adjusted', 'window_max', 'window_min']])


        # 결과 저장
        results_list.append(result)

        print(f"\n🔹 Step {step} 예측 결과:")
        print(result[['ds', 'y', 'yhat_adjusted']])

        mae = mean_absolute_error(result['y'], result['yhat_adjusted'])
        mse = mean_squared_error(result['y'], result['yhat_adjusted'])
        rmse = np.sqrt(mse)
        smape_value = smape(result['y'], result['yhat_adjusted'])

        print(f"\n🔹 Step {step} 평가:")
        print(f"MAE: {mae:.2f}, MSE: {mse:.2f}, RMSE: {rmse:.2f}, SMAPE: {smape_value:.2f}%\n")

        category_metrics[f"step{step}"] = {
            "MAE": mae, "MSE": mse, "RMSE": rmse, "SMAPE": smape_value
        }

    metrics[category] = category_metrics

# -------------------------------
# 평가 지표 저장 (Excel)
# -------------------------------
df_metrics = pd.DataFrame.from_dict({(i, j): metrics[i][j]
                                     for i in metrics.keys()
                                     for j in metrics[i].keys()}, orient='index')

df_metrics.to_excel("evaluation_metrics.xlsx")
print("✅ 평가 지표 저장 완료: evaluation_metrics.xlsx")
print(df_metrics)


# -------------------------------
# 최종 결과 출력
# -------------------------------
df_results = pd.concat(results_list, ignore_index=True)
print(df_results[['ds','item_id',  'y', 'yhat', 'yhat_adjusted']])

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 5.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpncgd2x7f/i8rgj6xb.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpncgd2x7f/25zjr3uf.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=93023', 'data', 'file=/tmp/tmpncgd2x7f/i8rgj6xb.json', 'init=/tmp/tmpncgd2x7f/25zjr3uf.json', 'output', 'file=/tmp/tmpncgd2x7f/prophet_models62jy9ou/prophet_model-20250301084641.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
08:46:41 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


📌 카테고리: 교통


08:46:41 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 2.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpncgd2x7f/9c6nuhri.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpncgd2x7f/9hvhgwzi.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=9393', 'data', 'file=/tmp/tmpncgd2x7f/9c6nuhri.json', 'init=/tmp/tmpncgd2x7f/9hvhgwzi.json', 'output', 'file=/tmp/tmpncgd2x7f/prophet_modellbmbsjo7/prophet_model-20250301084641.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
08:46:41 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


사용 기간: 2024-07-08 00:00:00 ~ 2024-09-02 00:00:00 (총 56일)
계산된 window_min: 0.0, window_max: 150950.0, rolling_mean: 33706.35

🔹 Step 1 예측 결과:
          ds      y  yhat_adjusted
0 2024-09-02  45200         435099

🔹 Step 1 평가:
MAE: 389899.23, MSE: 152021410047.34, RMSE: 389899.23, SMAPE: 162.36%



08:46:43 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 0.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpncgd2x7f/i8vo1_xy.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpncgd2x7f/lp1fodkv.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=54603', 'data', 'file=/tmp/tmpncgd2x7f/i8vo1_xy.json', 'init=/tmp/tmpncgd2x7f/lp1fodkv.json', 'output', 'file=/tmp/tmpncgd2x7f/prophet_modelq1dt9roj/prophet_model-20250301084643.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
08:46:43 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


...

사용 기간: 2025-01-13 00:00:00 ~ 2025-01-27 00:00:00 (총 14일)
계산된 window_min: 0.0, window_max: 157696.75, rolling_mean: 53535.425

🔹 Step 3 예측 결과:
          ds      y  yhat_adjusted
0 2025-01-27  32141         147271
1 2025-02-03 161383         140074

🔹 Step 3 평가:
MAE: 68219.73, MSE: 6854550197.85, RMSE: 82792.21, SMAPE: 71.24%

📌 카테고리: 카페/간식
사용 기간: 2024-07-08 00:00:00 ~ 2024-09-02 00:00:00 (총 56일)
계산된 window_min: 0.0, window_max: 150950.0, rolling_mean: 33706.35

🔹 Step 1 예측 결과:
          ds  y  yhat_adjusted
0 2024-09-02  0         128139

🔹 Step 1 평가:
MAE: 128139.26, MSE: 16419670859.51, RMSE: 128139.26, SMAPE: 200.00%



08:46:56 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 0.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpncgd2x7f/xw6ezl9k.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpncgd2x7f/mi8wjxie.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=70910', 'data', 'file=/tmp/tmpncgd2x7f/xw6ezl9k.json', 'init=/tmp/tmpncgd2x7f/mi8wjxie.json', 'output', 'file=/tmp/tmpncgd2x7f/prophet_model1kfokags/prophet_model-20250301084656.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
08:46:56 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


사용 기간: 2024-09-09 00:00:00 ~ 2024-10-07 00:00:00 (총 28일)
계산된 window_min: 0.0, window_max: 224562.5, rolling_mean: 69017.5

🔹 Step 2 예측 결과:
          ds      y  yhat_adjusted
0 2024-10-07  36770              0

🔹 Step 2 평가:
MAE: 36770.00, MSE: 1352032900.00, RMSE: 36770.00, SMAPE: 200.00%



08:46:56 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


사용 기간: 2025-01-13 00:00:00 ~ 2025-01-27 00:00:00 (총 14일)
계산된 window_min: 0.0, window_max: 157696.75, rolling_mean: 53535.425

🔹 Step 3 예측 결과:
          ds     y  yhat_adjusted
0 2025-01-27     0         381913
1 2025-02-03  5900        6718087

🔹 Step 3 평가:
MAE: 3547049.94, MSE: 22599652762905.91, RMSE: 4753909.21, SMAPE: 199.82%

✅ 평가 지표 저장 완료: evaluation_metrics.xlsx
                   MAE                 MSE       RMSE  SMAPE
교통    step1     389899        152021410047     389899    162
      step2      78242          6121888619      78242    178
      step3     183531         54910732600     234330    107
문화/여가 step1     378132        142983627261     378132    178
      step2 1431088014 2048012904684134144 1431088014    200
      step3       6750            56125000       7492    200
식비    step1      44747          2002313728      44747     37
      step2      23500           552250000      23500    200
      step3      60553          6729711214      82035     90
온라인쇼핑 step1      1

다른 모델과 비교- 일별 데이터 기반 앙상블 딥러닝